In [1]:
import numpy as np

from pol_analysis.polData import PolData
from pol_analysis.runPhot import RunPhot

In [2]:
def run_all(obj, band, star_coords, stddev_0_vals=None, show_seeing_plots=False, pdata_force_new=False, phot_force_new=False, centroid_box_size=5, pdata_filenames_for_masks=None):

    #Star coords for PSF measurements
    star_ex_ref, star_ey_ref = star_coords

    #Create the data object. This will run the mask creation and the cosmic ray rejection.  
    bhd = PolData(obj, band, filenames_for_mask_creation=pdata_filenames_for_masks, force_new=pdata_force_new)

    #Create the photometry object and run the common steps. 
    phot = RunPhot(bhd, force_new=phot_force_new)
    phot.subtract_background(box_size=(25,25))
    phot.get_source_positions(box_size=centroid_box_size)

    #Get the seeing. 
    fnames = bhd.list_of_filenames(ob_ids=None, mjds=None)
    if stddev_0_vals is not None:
        stddev_0 = stddev_0_vals
    else:
        stddev_0 = np.ones(len(fnames))*1.1
    phot.find_seeing(star_ex_ref, star_ey_ref, stddev_0=stddev_0, show_plots=show_seeing_plots, ob_ids=None, mjds=None)

    output_lines = list()
    for i, fname in enumerate(fnames):
        output_lines.append("{} {} {:.2f} {}\n".format(obj, band, phot.seeing[i], fname))

    return output_lines



In [3]:
objects = [
    "W0019-1046",
    "W0116-0505",
    "W0204-0506",
    "W0220+0137",
    "W0831+0140",
]
bands = dict()
for obj in objects:
    bands[obj] = ["R_SPECIAL"]
    if obj=="W0116-0505":
        bands[obj].extend(["I_BESS","v_HIGH"])

In [4]:
star_coords = dict()
cat = open("reference_stars.txt")
for line in cat:
    x = line.split()
    star_coords[x[0]] = (float(x[1]), float(x[2]))
cat.close()
# star_coords = {
#     "W0019-1046": (1470.5,  84.0),
#     "W0204-0506": (1086.5, 222.5),
#     "W0220+0137": (1600.0, 262.0),
#     "W0831+0140": (1010.0, 404.0),
#     "W0116-0505": (1398.0, 413.0),
# }

In [6]:
#run_all("W0019-1046", "R_SPECIAL", star_coords=(1145, 37.5))
cato = open("All_seeing_values.txt", "w")
for obj in objects:
    for band in bands[obj]:
        output_lines = run_all(obj, band, star_coords=star_coords[obj])
        for output_line in output_lines:
            cato.write(output_line)
        break
    break
cato.close()

mkdir: work: File exists
